In [2]:
from utils import init

init()

In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [4]:
search = TavilySearchResults()

In [5]:
search.invoke("what is the weather in SF")

[{'url': 'https://www.weatherapi.com/',
  'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1739770554, 'localtime': '2025-02-16 21:35'}, 'current': {'last_updated_epoch': 1739770200, 'last_updated': '2025-02-16 21:30', 'temp_c': 12.2, 'temp_f': 54.0, 'is_day': 0, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/night/122.png', 'code': 1009}, 'wind_mph': 4.5, 'wind_kph': 7.2, 'wind_degree': 290, 'wind_dir': 'WNW', 'pressure_mb': 1020.0, 'pressure_in': 30.11, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 97, 'cloud': 100, 'feelslike_c': 11.7, 'feelslike_f': 53.1, 'windchill_c': 11.2, 'windchill_f': 52.2, 'heatindex_c': 11.3, 'heatindex_f': 52.4, 'dewpoint_c': 10.9, 'dewpoint_f': 51.6, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 0.0, 'gust_mph': 6.9, 'gust_kph': 11.2}}"},
 {'url': 'https://weathershogun.com/weather/

In [7]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

In [8]:
retriever.invoke("how to upload a dataset")[0]

Document(id='f6891146-b502-452c-9836-b0ae2b123b7e', metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications.', 'language': 'en'}, page_content='Get started by adding tracing to your application.\nCreate dashboards to view key metrics like RPS, error rates and costs.\n\nEvals\u200b\nThe quality and development speed of AI applications depends on high-quality evaluation datasets and metrics to test and optimize your applications on. The LangSmith SDK and UI make building and running high-quality evaluations easy.\n\nGet started by creating your first evaluation.\nQuickly assess the performance of your application using our off-the-shelf evaluators (Python only) as a starting point.\nAnalyze results of evaluations in the LangSmith UI and compare results over time.\nEasily collect human feedback on your data to improve your application.\n\n

In [9]:
from langchain.tools.retriever import create_retriever_tool

In [10]:
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [11]:
tools = [search, retriever_tool]

In [12]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

In [13]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [14]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, prompt)

In [15]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [16]:
agent_executor.invoke({"input": "hi!"})



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


{'input': 'hi!', 'output': 'Hello! How can I assist you today?'}

In [17]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'how can LangSmith help with testing'}`


LangSmith + LangChain OSSLangSmith integrates seamlessly with LangChain's open source frameworks langchain and langgraph, with no extra instrumentation needed.If you're already using either of these, see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.
Observability​
Observability is important for any software application, but especially so for LLM applications. LLMs are non-deterministic by nature, meaning they can produce unexpected results. This makes them trickier than normal to debug.
This is where LangSmith can help! LangSmith has LLM-native observability, allowing you to get meaningful insights from your application. LangSmith’s observability features have you covered throughout all stages of application development - from prototyping, to beta testing, to production.

Get started with LangSmith | 🦜️🛠️ Lang

{'input': 'how can langsmith help with testing?',
 'output': "LangSmith can help with testing in the following ways:\n\n1. **Observability**: LangSmith provides LLM-native observability, allowing you to get meaningful insights from your application. This observability feature covers all stages of application development, from prototyping to beta testing to production.\n\n2. **Evaluations**: LangSmith offers tools for evaluating your application over production traffic, scoring application performance, and receiving human feedback on your data. The LangSmith SDK and UI make building and running high-quality evaluations easy.\n\n3. **Prompt Engineering**: AI applications involve writing prompts to instruct the LLM on what to do. LangSmith provides tools for prompt engineering, enabling you to find the perfect prompt for your application.\n\nBy utilizing LangSmith's observability, evaluation, and prompt engineering features, you can test and optimize your applications effectively."}

In [18]:
agent_executor.invoke({"input": "whats the weather in sf?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`


[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1739770795, 'localtime': '2025-02-16 21:39'}, 'current': {'last_updated_epoch': 1739770200, 'last_updated': '2025-02-16 21:30', 'temp_c': 12.2, 'temp_f': 54.0, 'is_day': 0, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/night/122.png', 'code': 1009}, 'wind_mph': 4.5, 'wind_kph': 7.2, 'wind_degree': 290, 'wind_dir': 'WNW', 'pressure_mb': 1020.0, 'pressure_in': 30.11, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 97, 'cloud': 100, 'feelslike_c': 11.7, 'feelslike_f': 53.1, 'windchill_c': 11.2, 'windchill_f': 52.2, 'heatindex_c': 11.3, 'heatindex_f': 52.4, 'dewpoint_c': 10.9, 'dewpoint_f': 51.6, 'vi

{'input': 'whats the weather in sf?',
 'output': 'The current weather in San Francisco is as follows:\n- Temperature: 12.2°C (54.0°F)\n- Condition: Overcast\n- Wind: 4.5 mph from WNW\n- Humidity: 97%\n- Cloud Cover: 100%\n- Feels like: 11.7°C (53.1°F)\n\nFor more detailed information, you can visit [Weather API](https://www.weatherapi.com/).'}

# Adding Memory

In [19]:
# Here we pass in an empty list of messages for chat_history because it is the first message in the chat
agent_executor.invoke({"input": "hi! my name is bob", "chat_history": []})



> Entering new AgentExecutor chain...
Hello Bob! How can I assist you today?

> Finished chain.


{'input': 'hi! my name is bob',
 'chat_history': [],
 'output': 'Hello Bob! How can I assist you today?'}

In [20]:
from langchain_core.messages import AIMessage, HumanMessage

In [21]:
agent_executor.invoke(
    {
        "chat_history": [
            HumanMessage(content="hi! my name is bob"),
            AIMessage(content="Hello Bob! How can I assist you today?"),
        ],
        "input": "what's my name?",
    }
)



> Entering new AgentExecutor chain...
Your name is Bob. How can I help you, Bob?

> Finished chain.


{'chat_history': [HumanMessage(content='hi! my name is bob', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello Bob! How can I assist you today?', additional_kwargs={}, response_metadata={})],
 'input': "what's my name?",
 'output': 'Your name is Bob. How can I help you, Bob?'}

In [22]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [23]:
message_history = ChatMessageHistory()

In [24]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [26]:
agent_with_chat_history.invoke(
    {"input": "hi! I'm bob"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "12333"}},
)



> Entering new AgentExecutor chain...
Hello Bob! How can I assist you today?

> Finished chain.


{'input': "hi! I'm bob",
 'chat_history': [HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello Bob! How can I assist you today?', additional_kwargs={}, response_metadata={})],
 'output': 'Hello Bob! How can I assist you today?'}

In [27]:
agent_with_chat_history.invoke(
    {"input": "what's my name?"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "222"}},
)



> Entering new AgentExecutor chain...
Your name is Bob! How can I help you, Bob?

> Finished chain.


{'input': "what's my name?",
 'chat_history': [HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello Bob! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello Bob! How can I assist you today?', additional_kwargs={}, response_metadata={})],
 'output': 'Your name is Bob! How can I help you, Bob?'}